# Intro

This notebook is for testing out the interactions between `pandas` and `IPython.display` as a lead-in to `dx` package development.

Currently established functionality:
- [establishing formatter baseline / deregistering DEX mediatype formatter](#?cellID=4a8ff57f-5b5f-46b6-ac4a-c12bf1f410b4)

> Resources
> ---
> - [**DEX Data Link Tech Design Doc**](https://docs.google.com/document/d/1KMJU2h7_cmL_wD84-SsGNVuIEI6-KmKchSEjKXnP85U/edit#)
> - [transport layer notebook](https://app.noteable-integration.us/f/a70232e4-4081-4f7a-86ad-fc59d8b21c4a)
> - [`geas` Data Processing Worker PR](https://github.com/noteable-io/geas/pull/1969)

> References
> ---
> - `pandas`
>   - [IPython formatter updating](https://github.com/pandas-dev/pandas/blob/ad190575aa75962d2d0eade2de81a5fe5a2e285b/pandas/io/formats/printing.py#L244)
>   - [DataFrame rendering](https://github.com/pandas-dev/pandas/blob/a8968bfa696d51f73769c54f2630a9530488236a/pandas/io/formats/format.py#L1003)
>       - [`DataFrame._repr_html_`](https://github.com/pandas-dev/pandas/blob/f9762d8f52e044d6588d6ee065952aa7e2d5362b/pandas/core/frame.py#L1025)
>       - [`NotebookFormatter`](https://github.com/pandas-dev/pandas/blob/a8968bfa696d51f73769c54f2630a9530488236a/pandas/io/formats/html.py#L585)


In [1]:
%pip install pandas

You should consider upgrading via the '/home/djshoup/.pyenv/versions/3.9.7/envs/gate-3.9.7/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# there's probably a better way of checking here
try:
    from IPython import get_ipython
    if "IPKernelApp" in get_ipython().config:
        IN_NOTEBOOK_ENV = True
except ImportError:
    IN_NOTEBOOK_ENV = False


import os
from typing import Optional

from IPython.core.formatters import BaseFormatter
from IPython.display import display as ipydisplay  # because we want to test 'dx.display' later

from pandas.io.json import build_table_schema
import pandas as pd


DEX_MEDIA_TYPE = "application/vnd.dex.v1+json"
DATARESOURCE_MEDIA_TYPE = "application/vnd.dataresource+json"

# maybe we want to check this in `__init__.py` before updating IPython.display formatters?
NOTEABLE_ENV = 'NTBL_USER_ID' in os.environ

print(f"{IN_NOTEBOOK_ENV=}")
print(f"{NOTEABLE_ENV=}")

IN_NOTEBOOK_ENV=True
NOTEABLE_ENV=False


# Make some sample data

In [3]:
import pprint
import random
import string

n_rows = 100
data = {
    'numcol_1': [random.randint(0, 100) for _ in range(n_rows)],
    'numcol_2': [random.random() for _ in range(n_rows)],
    'numcol_3': [random.randint(-10, 10) * random.random() for _ in range(n_rows)],
    'bool_col': [random.choice([True, False]) for _ in range(n_rows)],
    'str_col': [''.join(random.sample(string.ascii_uppercase, 3)) for _ in range(n_rows)],
}
data.keys()

dict_keys(['numcol_1', 'numcol_2', 'numcol_3', 'bool_col', 'str_col'])

# Original `pd.DataFrame` formatting

ideally we would run this:

```python
from pandas.io.formats.printing import enable_data_resource_formatter
enable_data_resource_formatter(True)
```

and make sure we see the following in `get_ipython().display_formatter.formatters`:
```python
'application/vnd.dataresource+json': <pandas.io.formats.printing.enable_data_resource_formatter.<locals>.TableSchemaFormatter at 0x7f1b9983b850>
```

but that doesn't work after `register_dx_formatter()` is called

In [4]:
def deregister_dx_formatting(media_type: str = DEX_MEDIA_TYPE) -> None:
    """Reverts IPython.display_formatter.formatters to original states"""
    pd.options.display.html.table_schema = False
    pd.options.display.max_rows = 50_001
    
    formatters = get_ipython().display_formatter.formatters
    if media_type in formatters:
        formatters.pop(media_type)
        
    # not sure why using `pandas.io.formats.printing.enable_data_resource_formatter(True)` doesn't work as expected here
    from IPython.core.formatters import BaseFormatter
    class TableSchemaFormatter(BaseFormatter):
        print_method = "_repr_data_resource_"
        _return_type = (dict,)
    formatters[DATARESOURCE_MEDIA_TYPE] = TableSchemaFormatter()
    formatters[DATARESOURCE_MEDIA_TYPE].enabled = True

In [5]:
# deregister_dx_formatting()
# get_ipython().display_formatter.formatters

In [6]:
# if the output below doesn't show the default pandas->DEX view, something is wrong 
# and you may need to run the deregister function above
df = pd.DataFrame(data)
df.sample(5)

,numcol_1,numcol_2,numcol_3,bool_col,str_col
10,72,0.162265,2.933297,False,BJO
21,4,0.508288,-0.494598,True,VQE
64,27,0.631291,-1.341139,False,ZJO
2,19,0.953691,0.000000,False,WUL
75,47,0.461670,6.307645,False,CUQ


In [7]:
# save off a file in case we want to try rendering direct-from-file later
df.to_csv("dx_sample_data.csv", index=False)

# Register the new IPython formatter and media type

we should see something like this included in `get_ipython().display_formatter.formatters`:
```python
'application/vnd.dex.v1+json': <__main__.DEXDataFrameFormatter at 0x7f1b9963ac40>
```

In [8]:
class DEXDataFrameFormatter(BaseFormatter):
    print_method = "_repr_data_resource_"
    _return_type = (dict,)
    
    
def register_dx_formatter(media_type: str = DEX_MEDIA_TYPE) -> None:
    """Registers a mediatype for IPython display formatting""" 
    pd.options.display.html.table_schema = True
    pd.options.display.max_rows = 100_000
    
    formatters = get_ipython().display_formatter.formatters     
    # add new DEX formatter
    formatters[media_type] = DEXDataFrameFormatter()
    formatters[media_type].enabled = False  # this breaks default Dataframe._repl_html_ if enabled

    
register_dx_formatter()
get_ipython().display_formatter.formatters 

{'text/plain': <IPython.core.formatters.PlainTextFormatter at 0x7fe26cff7070>,
 'text/html': <IPython.core.formatters.HTMLFormatter at 0x7fe26cff7e80>,
 'text/markdown': <IPython.core.formatters.MarkdownFormatter at 0x7fe26cff7520>,
 'image/svg+xml': <IPython.core.formatters.SVGFormatter at 0x7fe26cff7700>,
 'image/png': <IPython.core.formatters.PNGFormatter at 0x7fe26cff7ac0>,
 'application/pdf': <IPython.core.formatters.PDFFormatter at 0x7fe26cff7af0>,
 'image/jpeg': <IPython.core.formatters.JPEGFormatter at 0x7fe26cff7730>,
 'text/latex': <IPython.core.formatters.LatexFormatter at 0x7fe26cff7400>,
 'application/json': <IPython.core.formatters.JSONFormatter at 0x7fe26cff7be0>,
 'application/javascript': <IPython.core.formatters.JavascriptFormatter at 0x7fe26cff74c0>,
 'application/vnd.dataresource+json': <pandas.io.formats.printing.enable_data_resource_formatter.<locals>.TableSchemaFormatter at 0x7fe22e4cc2b0>,
 'application/vnd.dex.v1+json': <__main__.DEXDataFrameFormatter at 0x7fe2

In [9]:
# if we configured everything above properly, 
# the default pandas display should be using the dex.v1 media type
df

,numcol_1,numcol_2,numcol_3,bool_col,str_col
0,74,0.191360,5.695822,False,JPR
1,64,0.101241,-5.320792,True,EWM
2,19,0.953691,0.000000,False,WUL
3,68,0.979403,0.408580,False,VFB
4,90,0.594331,-8.534415,True,EBG
5,20,0.789158,-0.651252,False,PYX
6,4,0.989974,0.095438,False,SVN
7,9,0.045342,-3.168897,True,ZOH
8,80,0.491779,0.843663,False,LRA
9,66,0.025786,6.974413,False,EHI


# Subclass `pd.DataFrame` and start providing convenience methods for DEX displays

this is definitely a work in progress and subject to tons of change

In [20]:
# toggle this to show/hide the payloads sent to `IPython.display`
SHOW_PAYLOADS = False

In [21]:
from IPython.display import Markdown

class DEXDataFrame(pd.DataFrame):
    """Subclass DataFrame to provide DEX-focused convenience methods"""
    
    _display_index = False
    media_type = DEX_MEDIA_TYPE
    
    def display(self, media_type: Optional[str] = None, index: bool = False):
        """Render DEXDataFrame based on provided media type."""
        media_type = media_type or self.media_type
        self._display_index = index
        
        if media_type != self.media_type:
            # non-default media type; use default orient
            data = self.to_dict(orient="records")
        else:
            # we can't use `.to_dict(orient='list')` here since that would return a dictionary of {column: [values]} pairs
            transform = self.transpose
            if self._display_index:
                transform = self.reset_index().transpose
            data = transform().values.tolist()
            
        payload = {
            "schema": self.table_schema,
            "data": data,
            #"summary_statistics": {},
            #"dx-seed": {},
        }
        
        ### ONLY FOR TESTING - REMOVE THIS LATER ###
        if SHOW_PAYLOADS:
            ipydisplay(Markdown(f"""sending the following to `IPython.display.display` :
```python
{pprint.pformat({media_type: payload})}
```"""))
        ### --- ###
        
        ipydisplay({media_type: payload}, raw=True)
        return
    
    @property
    def table_schema(self):
        return build_table_schema(self, index=self._display_index)
    
    
def display(df: pd.DataFrame, media_type: Optional[str] = None, index: bool = False) -> None:
    """Convenience function to allow calling `dx.display(df)` on a pandas Dataframe."""
    
    # TODO: handle when `df` isn't an instance of `pd.DataFrame`
    # - read files? 
    # - pd.json_normalize()?
    # - ...other data parsing?
    # should be imported from `dx/__init__.py` to allow `dx.display(df)`
    
    return DEXDataFrame(df).display(media_type=media_type, index=index)

# Now let's see if `display(df)` works

## first test a `pd.DataFrame`

In [22]:
df = pd.DataFrame(data)
df

,numcol_1,numcol_2,numcol_3,bool_col,str_col
0,74,0.191360,5.695822,False,JPR
1,64,0.101241,-5.320792,True,EWM
2,19,0.953691,0.000000,False,WUL
3,68,0.979403,0.408580,False,VFB
4,90,0.594331,-8.534415,True,EBG
5,20,0.789158,-0.651252,False,PYX
6,4,0.989974,0.095438,False,SVN
7,9,0.045342,-3.168897,True,ZOH
8,80,0.491779,0.843663,False,LRA
9,66,0.025786,6.974413,False,EHI


In [23]:
display(df, media_type=DATARESOURCE_MEDIA_TYPE)

In [24]:
display(df)  # media_type=DEX_MEDIA_TYPE by default

## test with `DEXDataFrame` subclass
...which ultimately shouldn't make any difference for now

In [17]:
# ...and using the `dx.display()` should render the same
ddf = DEXDataFrame(data)
ddf

,numcol_1,numcol_2,numcol_3,bool_col,str_col
0,74,0.191360,5.695822,False,JPR
1,64,0.101241,-5.320792,True,EWM
2,19,0.953691,0.000000,False,WUL
3,68,0.979403,0.408580,False,VFB
4,90,0.594331,-8.534415,True,EBG
5,20,0.789158,-0.651252,False,PYX
6,4,0.989974,0.095438,False,SVN
7,9,0.045342,-3.168897,True,ZOH
8,80,0.491779,0.843663,False,LRA
9,66,0.025786,6.974413,False,EHI


In [18]:
display(ddf, media_type=DATARESOURCE_MEDIA_TYPE)

sending the following to `IPython.display.display` :
```python
{'application/vnd.dataresource+json': {'data': [{'bool_col': False,
                                                 'numcol_1': 74,
                                                 'numcol_2': 0.19136036918636945,
                                                 'numcol_3': 5.69582151791086,
                                                 'str_col': 'JPR'},
                                                {'bool_col': True,
                                                 'numcol_1': 64,
                                                 'numcol_2': 0.10124147145336515,
                                                 'numcol_3': -5.320791875001649,
                                                 'str_col': 'EWM'},
                                                {'bool_col': False,
                                                 'numcol_1': 19,
                                                 'numcol_2': 0.9536914680579314,
                                                 'numcol_3': 0.0,
                                                 'str_col': 'WUL'},
                                                {'bool_col': False,
                                                 'numcol_1': 68,
                                                 'numcol_2': 0.9794029938689203,
                                                 'numcol_3': 0.4085795650820677,
                                                 'str_col': 'VFB'},
                                                {'bool_col': True,
                                                 'numcol_1': 90,
                                                 'numcol_2': 0.59433107517774,
                                                 'numcol_3': -8.534415361809245,
                                                 'str_col': 'EBG'},
                                                {'bool_col': False,
                                                 'numcol_1': 20,
                                                 'numcol_2': 0.7891584619489574,
                                                 'numcol_3': -0.6512524508932834,
                                                 'str_col': 'PYX'},
                                                {'bool_col': False,
                                                 'numcol_1': 4,
                                                 'numcol_2': 0.9899737264206632,
                                                 'numcol_3': 0.0954383181602585,
                                                 'str_col': 'SVN'},
                                                {'bool_col': True,
                                                 'numcol_1': 9,
                                                 'numcol_2': 0.04534155466338696,
                                                 'numcol_3': -3.168897026057221,
                                                 'str_col': 'ZOH'},
                                                {'bool_col': False,
                                                 'numcol_1': 80,
                                                 'numcol_2': 0.4917789167627633,
                                                 'numcol_3': 0.8436626138152887,
                                                 'str_col': 'LRA'},
                                                {'bool_col': False,
                                                 'numcol_1': 66,
                                                 'numcol_2': 0.02578605230060127,
                                                 'numcol_3': 6.974413169201763,
                                                 'str_col': 'EHI'},
                                                {'bool_col': False,
                                                 'numcol_1': 72,
                                                 'numcol_2': 0.16226536572062777,
                                                 'numcol_3': 2.9332973995192924,
                                                 'str_col': 'BJO'},
                                                {'bool_col': False,
                                                 'numcol_1': 15,
                                                 'numcol_2': 0.5719925155839274,
                                                 'numcol_3': 0.0,
                                                 'str_col': 'IWL'},
                                                {'bool_col': True,
                                                 'numcol_1': 81,
                                                 'numcol_2': 0.4728690227143072,
                                                 'numcol_3': 2.494678891248557,
                                                 'str_col': 'IAB'},
                                                {'bool_col': True,
                                                 'numcol_1': 91,
                                                 'numcol_2': 0.9559278628821611,
                                                 'numcol_3': 3.187540392127616,
                                                 'str_col': 'UPM'},
                                                {'bool_col': True,
                                                 'numcol_1': 47,
                                                 'numcol_2': 0.29926601652555873,
                                                 'numcol_3': 4.390156296743939,
                                                 'str_col': 'KPH'},
                                                {'bool_col': False,
                                                 'numcol_1': 30,
                                                 'numcol_2': 0.343185849180506,
                                                 'numcol_3': 0.6703486138323687,
                                                 'str_col': 'CAV'},
                                                {'bool_col': False,
                                                 'numcol_1': 31,
                                                 'numcol_2': 0.6648658385769954,
                                                 'numcol_3': 0.18820299486742265,
                                                 'str_col': 'XLJ'},
                                                {'bool_col': True,
                                                 'numcol_1': 48,
                                                 'numcol_2': 0.6503475384450885,
                                                 'numcol_3': -2.8076521015770615,
                                                 'str_col': 'GVI'},
                                                {'bool_col': False,
                                                 'numcol_1': 76,
                                                 'numcol_2': 0.7712715630468518,
                                                 'numcol_3': -8.96649522083653,
                                                 'str_col': 'ZED'},
                                                {'bool_col': True,
                                                 'numcol_1': 46,
                                                 'numcol_2': 0.5380512979707743,
                                                 'numcol_3': -8.505276314875948,
                                                 'str_col': 'KMA'},
                                                {'bool_col': True,
                                                 'numcol_1': 77,
                                                 'numcol_2': 0.8875985140858073,
                                                 'numcol_3': 0.5445477463985948,
                                                 'str_col': 'PMG'},
                                                {'bool_col': True,
                                                 'numcol_1': 4,
                                                 'numcol_2': 0.5082878165418838,
                                                 'numcol_3': -0.49459832920734026,
                                                 'str_col': 'VQE'},
                                                {'bool_col': True,
                                                 'numcol_1': 44,
                                                 'numcol_2': 0.7348155575755595,
                                                 'numcol_3': 2.4788807005417945,
                                                 'str_col': 'KZV'},
                                                {'bool_col': False,
                                                 'numcol_1': 87,
                                                 'numcol_2': 0.11149743675519541,
                                                 'numcol_3': 0.295397906420525,
                                                 'str_col': 'PDI'},
                                                {'bool_col': False,
                                                 'numcol_1': 59,
                                                 'numcol_2': 0.2538474247453584,
                                                 'numcol_3': 0.08026277580341168,
                                                 'str_col': 'DKS'},
                                                {'bool_col': False,
                                                 'numcol_1': 11,
                                                 'numcol_2': 0.8860852907051684,
                                                 'numcol_3': 0.05375343408431654,
                                                 'str_col': 'DVC'},
                                                {'bool_col': True,
                                                 'numcol_1': 80,
                                                 'numcol_2': 0.05232792426318955,
                                                 'numcol_3': -7.935370762650371,
                                                 'str_col': 'GZM'},
                                                {'bool_col': True,
                                                 'numcol_1': 21,
                                                 'numcol_2': 0.4747450244038923,
                                                 'numcol_3': 5.841083911459567,
                                                 'str_col': 'LXF'},
                                                {'bool_col': True,
                                                 'numcol_1': 82,
                                                 'numcol_2': 0.8295796814498165,
                                                 'numcol_3': -0.8986440016617555,
                                                 'str_col': 'BTN'},
                                                {'bool_col': True,
                                                 'numcol_1': 37,
                                                 'numcol_2': 0.9966152810617562,
                                                 'numcol_3': -2.9517662582078956,
                                                 'str_col': 'CXJ'},
                                                {'bool_col': True,
                                                 'numcol_1': 76,
                                                 'numcol_2': 0.3874448916318989,
                                                 'numcol_3': 5.0026753740632515,
                                                 'str_col': 'KGO'},
                                                {'bool_col': False,
                                                 'numcol_1': 53,
                                                 'numcol_2': 0.8740717137968557,
                                                 'numcol_3': -4.905906849591305,
                                                 'str_col': 'MSP'},
                                                {'bool_col': True,
                                                 'numcol_1': 67,
                                                 'numcol_2': 0.4705292495099942,
                                                 'numcol_3': 7.847989996961754,
                                                 'str_col': 'QMV'},
                                                {'bool_col': False,
                                                 'numcol_1': 40,
                                                 'numcol_2': 0.4872675829371177,
                                                 'numcol_3': -0.9310582210169116,
                                                 'str_col': 'CWD'},
                                                {'bool_col': True,
                                                 'numcol_1': 16,
                                                 'numcol_2': 0.7986816367436043,
                                                 'numcol_3': 2.2781332156064513,
                                                 'str_col': 'LDJ'},
                                                {'bool_col': False,
                                                 'numcol_1': 73,
                                                 'numcol_2': 0.23031175492263156,
                                                 'numcol_3': 1.979175993616305,
                                                 'str_col': 'YRD'},
                                                {'bool_col': False,
                                                 'numcol_1': 77,
                                                 'numcol_2': 0.153076425667535,
                                                 'numcol_3': 3.2886809041405556,
                                                 'str_col': 'IDF'},
                                                {'bool_col': True,
                                                 'numcol_1': 63,
                                                 'numcol_2': 0.2305366083994328,
                                                 'numcol_3': -0.9606273597635134,
                                                 'str_col': 'RSC'},
                                                {'bool_col': False,
                                                 'numcol_1': 73,
                                                 'numcol_2': 0.16637494321900603,
                                                 'numcol_3': -2.0064036876723,
                                                 'str_col': 'GPB'},
                                                {'bool_col': True,
                                                 'numcol_1': 41,
                                                 'numcol_2': 0.3420086511813747,
                                                 'numcol_3': 4.387755267157399,
                                                 'str_col': 'XBD'},
                                                {'bool_col': False,
                                                 'numcol_1': 25,
                                                 'numcol_2': 0.6181527841925196,
                                                 'numcol_3': 4.088533270003084,
                                                 'str_col': 'TDA'},
                                                {'bool_col': True,
                                                 'numcol_1': 91,
                                                 'numcol_2': 0.8646883744735294,
                                                 'numcol_3': -8.360162284881437,
                                                 'str_col': 'PQM'},
                                                {'bool_col': True,
                                                 'numcol_1': 27,
                                                 'numcol_2': 0.0006604903737036372,
                                                 'numcol_3': 0.4618348742121261,
                                                 'str_col': 'IOY'},
                                                {'bool_col': True,
                                                 'numcol_1': 29,
                                                 'numcol_2': 0.445432666702669,
                                                 'numcol_3': 0.016062780287119383,
                                                 'str_col': 'GQT'},
                                                {'bool_col': True,
                                                 'numcol_1': 75,
                                                 'numcol_2': 0.17507523565845162,
                                                 'numcol_3': -3.6988353829878937,
                                                 'str_col': 'MLN'},
                                                {'bool_col': True,
                                                 'numcol_1': 86,
                                                 'numcol_2': 0.7006598292599899,
                                                 'numcol_3': 4.739396996105104,
                                                 'str_col': 'ADN'},
                                                {'bool_col': False,
                                                 'numcol_1': 53,
                                                 'numcol_2': 0.8947391975647094,
                                                 'numcol_3': 3.8327892474926424,
                                                 'str_col': 'XFY'},
                                                {'bool_col': True,
                                                 'numcol_1': 65,
                                                 'numcol_2': 0.08970369637153264,
                                                 'numcol_3': 1.4583759365804085,
                                                 'str_col': 'GAE'},
                                                {'bool_col': True,
                                                 'numcol_1': 51,
                                                 'numcol_2': 0.3420333985809034,
                                                 'numcol_3': 3.2554750361193805,
                                                 'str_col': 'IFY'},
                                                {'bool_col': True,
                                                 'numcol_1': 50,
                                                 'numcol_2': 0.6986815360362426,
                                                 'numcol_3': -0.6863185795243316,
                                                 'str_col': 'UZJ'},
                                                {'bool_col': True,
                                                 'numcol_1': 71,
                                                 'numcol_2': 0.5728279215993398,
                                                 'numcol_3': -2.0361564497949702,
                                                 'str_col': 'AHJ'},
                                                {'bool_col': True,
                                                 'numcol_1': 84,
                                                 'numcol_2': 0.18316537693229873,
                                                 'numcol_3': 2.375697644347084,
                                                 'str_col': 'NJR'},
                                                {'bool_col': True,
                                                 'numcol_1': 23,
                                                 'numcol_2': 0.5992383951763239,
                                                 'numcol_3': -1.0226647759803935,
                                                 'str_col': 'LRW'},
                                                {'bool_col': True,
                                                 'numcol_1': 70,
                                                 'numcol_2': 0.5066833960139401,
                                                 'numcol_3': -0.7341151374133368,
                                                 'str_col': 'GPZ'},
                                                {'bool_col': False,
                                                 'numcol_1': 83,
                                                 'numcol_2': 0.9948710378199194,
                                                 'numcol_3': -0.525337839865223,
                                                 'str_col': 'BKJ'},
                                                {'bool_col': False,
                                                 'numcol_1': 1,
                                                 'numcol_2': 0.557022247846315,
                                                 'numcol_3': 1.829047770157662,
                                                 'str_col': 'QZS'},
                                                {'bool_col': True,
                                                 'numcol_1': 78,
                                                 'numcol_2': 0.14144308392044724,
                                                 'numcol_3': -0.20252436510343086,
                                                 'str_col': 'ISU'},
                                                {'bool_col': True,
                                                 'numcol_1': 53,
                                                 'numcol_2': 0.39120394655037705,
                                                 'numcol_3': 5.005504671198587,
                                                 'str_col': 'RXL'},
                                                {'bool_col': True,
                                                 'numcol_1': 75,
                                                 'numcol_2': 0.053264045664866866,
                                                 'numcol_3': -6.254003563847993,
                                                 'str_col': 'FUJ'},
                                                {'bool_col': False,
                                                 'numcol_1': 38,
                                                 'numcol_2': 0.5660235889665234,
                                                 'numcol_3': 1.119520226839461,
                                                 'str_col': 'PKQ'},
                                                {'bool_col': False,
                                                 'numcol_1': 50,
                                                 'numcol_2': 0.7943247803108686,
                                                 'numcol_3': 0.4041274926695637,
                                                 'str_col': 'PFI'},
                                                {'bool_col': False,
                                                 'numcol_1': 43,
                                                 'numcol_2': 0.7432674802170094,
                                                 'numcol_3': 6.386115029082269,
                                                 'str_col': 'YCR'},
                                                {'bool_col': True,
                                                 'numcol_1': 75,
                                                 'numcol_2': 0.9263975967465905,
                                                 'numcol_3': 0.40785148105185254,
                                                 'str_col': 'BNA'},
                                                {'bool_col': False,
                                                 'numcol_1': 28,
                                                 'numcol_2': 0.1317675517914878,
                                                 'numcol_3': -3.6100703211197063,
                                                 'str_col': 'GPU'},
                                                {'bool_col': False,
                                                 'numcol_1': 27,
                                                 'numcol_2': 0.6312910466457831,
                                                 'numcol_3': -1.3411391785820288,
                                                 'str_col': 'ZJO'},
                                                {'bool_col': True,
                                                 'numcol_1': 20,
                                                 'numcol_2': 0.024985086708438353,
                                                 'numcol_3': -5.815052117335412,
                                                 'str_col': 'DUM'},
                                                {'bool_col': True,
                                                 'numcol_1': 4,
                                                 'numcol_2': 0.7661952705154852,
                                                 'numcol_3': 1.221917945462085,
                                                 'str_col': 'YES'},
                                                {'bool_col': True,
                                                 'numcol_1': 41,
                                                 'numcol_2': 0.5078098365978824,
                                                 'numcol_3': -8.372200012982317,
                                                 'str_col': 'EWI'},
                                                {'bool_col': True,
                                                 'numcol_1': 21,
                                                 'numcol_2': 0.6226409055062616,
                                                 'numcol_3': -0.8650172306990368,
                                                 'str_col': 'RZO'},
                                                {'bool_col': False,
                                                 'numcol_1': 39,
                                                 'numcol_2': 0.012816345941056184,
                                                 'numcol_3': 1.7224633030637484,
                                                 'str_col': 'PTV'},
                                                {'bool_col': True,
                                                 'numcol_1': 17,
                                                 'numcol_2': 0.16246493182418043,
                                                 'numcol_3': -0.6819785587583217,
                                                 'str_col': 'AWF'},
                                                {'bool_col': True,
                                                 'numcol_1': 7,
                                                 'numcol_2': 0.7084181113388888,
                                                 'numcol_3': 0.0,
                                                 'str_col': 'JSF'},
                                                {'bool_col': True,
                                                 'numcol_1': 5,
                                                 'numcol_2': 0.7426361934428682,
                                                 'numcol_3': -1.5590584882816314,
                                                 'str_col': 'TRV'},
                                                {'bool_col': True,
                                                 'numcol_1': 41,
                                                 'numcol_2': 0.48353938234650995,
                                                 'numcol_3': 0.0,
                                                 'str_col': 'XNA'},
                                                {'bool_col': True,
                                                 'numcol_1': 96,
                                                 'numcol_2': 0.20784398297675655,
                                                 'numcol_3': 0.0042756339138692034,
                                                 'str_col': 'MYX'},
                                                {'bool_col': False,
                                                 'numcol_1': 47,
                                                 'numcol_2': 0.4616696836683213,
                                                 'numcol_3': 6.307644672640099,
                                                 'str_col': 'CUQ'},
                                                {'bool_col': True,
                                                 'numcol_1': 21,
                                                 'numcol_2': 0.6435181164059954,
                                                 'numcol_3': 8.773288996733527,
                                                 'str_col': 'BCU'},
                                                {'bool_col': False,
                                                 'numcol_1': 65,
                                                 'numcol_2': 0.9288538111330569,
                                                 'numcol_3': -1.4889535227874742,
                                                 'str_col': 'QNB'},
                                                {'bool_col': True,
                                                 'numcol_1': 16,
                                                 'numcol_2': 5.074825244211212e-05,
                                                 'numcol_3': 3.1394560335760437,
                                                 'str_col': 'AYL'},
                                                {'bool_col': True,
                                                 'numcol_1': 94,
                                                 'numcol_2': 0.018762951979302556,
                                                 'numcol_3': 3.9506668467261354,
                                                 'str_col': 'XBF'},
                                                {'bool_col': True,
                                                 'numcol_1': 97,
                                                 'numcol_2': 0.34233166078223365,
                                                 'numcol_3': -2.3411123592187657,
                                                 'str_col': 'WPZ'},
                                                {'bool_col': True,
                                                 'numcol_1': 49,
                                                 'numcol_2': 0.34298905342366637,
                                                 'numcol_3': 1.69329033505479,
                                                 'str_col': 'WEM'},
                                                {'bool_col': True,
                                                 'numcol_1': 92,
                                                 'numcol_2': 0.41732863439581247,
                                                 'numcol_3': 0.0,
                                                 'str_col': 'XIS'},
                                                {'bool_col': False,
                                                 'numcol_1': 38,
                                                 'numcol_2': 0.18100449147540798,
                                                 'numcol_3': 4.505463965335339,
                                                 'str_col': 'PDK'},
                                                {'bool_col': False,
                                                 'numcol_1': 33,
                                                 'numcol_2': 0.34969609825442793,
                                                 'numcol_3': -4.793207203672118,
                                                 'str_col': 'WTC'},
                                                {'bool_col': False,
                                                 'numcol_1': 76,
                                                 'numcol_2': 0.8524702210180213,
                                                 'numcol_3': -8.959847454473886,
                                                 'str_col': 'ZMC'},
                                                {'bool_col': False,
                                                 'numcol_1': 25,
                                                 'numcol_2': 0.6828795712175348,
                                                 'numcol_3': 0.2963783561939113,
                                                 'str_col': 'VDL'},
                                                {'bool_col': True,
                                                 'numcol_1': 63,
                                                 'numcol_2': 0.13372144690237153,
                                                 'numcol_3': 2.490154930097335,
                                                 'str_col': 'MOC'},
                                                {'bool_col': False,
                                                 'numcol_1': 89,
                                                 'numcol_2': 0.7855265304468747,
                                                 'numcol_3': -2.9772991980786427,
                                                 'str_col': 'ENL'},
                                                {'bool_col': True,
                                                 'numcol_1': 47,
                                                 'numcol_2': 0.46628810563832757,
                                                 'numcol_3': -0.10258805242017521,
                                                 'str_col': 'WSE'},
                                                {'bool_col': True,
                                                 'numcol_1': 61,
                                                 'numcol_2': 0.8693902187754182,
                                                 'numcol_3': 2.515040437144264,
                                                 'str_col': 'TKZ'},
                                                {'bool_col': False,
                                                 'numcol_1': 74,
                                                 'numcol_2': 0.9639249912124923,
                                                 'numcol_3': -0.323752470193159,
                                                 'str_col': 'LGU'},
                                                {'bool_col': False,
                                                 'numcol_1': 72,
                                                 'numcol_2': 0.7093216353825577,
                                                 'numcol_3': -1.3871022557850115,
                                                 'str_col': 'KVC'},
                                                {'bool_col': False,
                                                 'numcol_1': 1,
                                                 'numcol_2': 0.8635913789031773,
                                                 'numcol_3': 0.09637447265227139,
                                                 'str_col': 'DEG'},
                                                {'bool_col': False,
                                                 'numcol_1': 0,
                                                 'numcol_2': 0.30572863549305196,
                                                 'numcol_3': 7.854363178840153,
                                                 'str_col': 'GSH'},
                                                {'bool_col': True,
                                                 'numcol_1': 14,
                                                 'numcol_2': 0.5818091800872877,
                                                 'numcol_3': 3.96354237781416,
                                                 'str_col': 'XBV'},
                                                {'bool_col': False,
                                                 'numcol_1': 84,
                                                 'numcol_2': 0.4005035087304083,
                                                 'numcol_3': -0.8107125934130455,
                                                 'str_col': 'LMH'},
                                                {'bool_col': True,
                                                 'numcol_1': 91,
                                                 'numcol_2': 0.07274119308832105,
                                                 'numcol_3': 0.7476953283759489,
                                                 'str_col': 'YFU'},
                                                {'bool_col': True,
                                                 'numcol_1': 63,
                                                 'numcol_2': 0.695356315467362,
                                                 'numcol_3': -0.8673789320239975,
                                                 'str_col': 'UEO'},
                                                {'bool_col': True,
                                                 'numcol_1': 99,
                                                 'numcol_2': 0.0905157809419529,
                                                 'numcol_3': -1.5850387856268093,
                                                 'str_col': 'KNG'}],
                                       'schema': {'fields': [{'name': 'numcol_1',
                                                              'type': 'integer'},
                                                             {'name': 'numcol_2',
                                                              'type': 'number'},
                                                             {'name': 'numcol_3',
                                                              'type': 'number'},
                                                             {'name': 'bool_col',
                                                              'type': 'boolean'},
                                                             {'name': 'str_col',
                                                              'type': 'string'}],
                                                  'pandas_version': '1.4.0'}}}
```

In [19]:
display(ddf)

sending the following to `IPython.display.display` :
```python
{'application/vnd.dex.v1+json': {'data': [[74,
                                           64,
                                           19,
                                           68,
                                           90,
                                           20,
                                           4,
                                           9,
                                           80,
                                           66,
                                           72,
                                           15,
                                           81,
                                           91,
                                           47,
                                           30,
                                           31,
                                           48,
                                           76,
                                           46,
                                           77,
                                           4,
                                           44,
                                           87,
                                           59,
                                           11,
                                           80,
                                           21,
                                           82,
                                           37,
                                           76,
                                           53,
                                           67,
                                           40,
                                           16,
                                           73,
                                           77,
                                           63,
                                           73,
                                           41,
                                           25,
                                           91,
                                           27,
                                           29,
                                           75,
                                           86,
                                           53,
                                           65,
                                           51,
                                           50,
                                           71,
                                           84,
                                           23,
                                           70,
                                           83,
                                           1,
                                           78,
                                           53,
                                           75,
                                           38,
                                           50,
                                           43,
                                           75,
                                           28,
                                           27,
                                           20,
                                           4,
                                           41,
                                           21,
                                           39,
                                           17,
                                           7,
                                           5,
                                           41,
                                           96,
                                           47,
                                           21,
                                           65,
                                           16,
                                           94,
                                           97,
                                           49,
                                           92,
                                           38,
                                           33,
                                           76,
                                           25,
                                           63,
                                           89,
                                           47,
                                           61,
                                           74,
                                           72,
                                           1,
                                           0,
                                           14,
                                           84,
                                           91,
                                           63,
                                           99],
                                          [0.19136036918636945,
                                           0.10124147145336515,
                                           0.9536914680579314,
                                           0.9794029938689203,
                                           0.59433107517774,
                                           0.7891584619489574,
                                           0.9899737264206632,
                                           0.04534155466338696,
                                           0.4917789167627633,
                                           0.02578605230060127,
                                           0.16226536572062777,
                                           0.5719925155839274,
                                           0.4728690227143072,
                                           0.9559278628821611,
                                           0.29926601652555873,
                                           0.343185849180506,
                                           0.6648658385769954,
                                           0.6503475384450885,
                                           0.7712715630468518,
                                           0.5380512979707743,
                                           0.8875985140858073,
                                           0.5082878165418838,
                                           0.7348155575755595,
                                           0.11149743675519541,
                                           0.2538474247453584,
                                           0.8860852907051684,
                                           0.05232792426318955,
                                           0.4747450244038923,
                                           0.8295796814498165,
                                           0.9966152810617562,
                                           0.3874448916318989,
                                           0.8740717137968557,
                                           0.4705292495099942,
                                           0.4872675829371177,
                                           0.7986816367436043,
                                           0.23031175492263156,
                                           0.153076425667535,
                                           0.2305366083994328,
                                           0.16637494321900603,
                                           0.3420086511813747,
                                           0.6181527841925196,
                                           0.8646883744735294,
                                           0.0006604903737036372,
                                           0.445432666702669,
                                           0.17507523565845162,
                                           0.7006598292599899,
                                           0.8947391975647094,
                                           0.08970369637153264,
                                           0.3420333985809034,
                                           0.6986815360362426,
                                           0.5728279215993398,
                                           0.18316537693229873,
                                           0.5992383951763239,
                                           0.5066833960139401,
                                           0.9948710378199194,
                                           0.557022247846315,
                                           0.14144308392044724,
                                           0.39120394655037705,
                                           0.053264045664866866,
                                           0.5660235889665234,
                                           0.7943247803108686,
                                           0.7432674802170094,
                                           0.9263975967465905,
                                           0.1317675517914878,
                                           0.6312910466457831,
                                           0.024985086708438353,
                                           0.7661952705154852,
                                           0.5078098365978824,
                                           0.6226409055062616,
                                           0.012816345941056184,
                                           0.16246493182418043,
                                           0.7084181113388888,
                                           0.7426361934428682,
                                           0.48353938234650995,
                                           0.20784398297675655,
                                           0.4616696836683213,
                                           0.6435181164059954,
                                           0.9288538111330569,
                                           5.074825244211212e-05,
                                           0.018762951979302556,
                                           0.34233166078223365,
                                           0.34298905342366637,
                                           0.41732863439581247,
                                           0.18100449147540798,
                                           0.34969609825442793,
                                           0.8524702210180213,
                                           0.6828795712175348,
                                           0.13372144690237153,
                                           0.7855265304468747,
                                           0.46628810563832757,
                                           0.8693902187754182,
                                           0.9639249912124923,
                                           0.7093216353825577,
                                           0.8635913789031773,
                                           0.30572863549305196,
                                           0.5818091800872877,
                                           0.4005035087304083,
                                           0.07274119308832105,
                                           0.695356315467362,
                                           0.0905157809419529],
                                          [5.69582151791086,
                                           -5.320791875001649,
                                           0.0,
                                           0.4085795650820677,
                                           -8.534415361809245,
                                           -0.6512524508932834,
                                           0.0954383181602585,
                                           -3.168897026057221,
                                           0.8436626138152887,
                                           6.974413169201763,
                                           2.9332973995192924,
                                           0.0,
                                           2.494678891248557,
                                           3.187540392127616,
                                           4.390156296743939,
                                           0.6703486138323687,
                                           0.18820299486742265,
                                           -2.8076521015770615,
                                           -8.96649522083653,
                                           -8.505276314875948,
                                           0.5445477463985948,
                                           -0.49459832920734026,
                                           2.4788807005417945,
                                           0.295397906420525,
                                           0.08026277580341168,
                                           0.05375343408431654,
                                           -7.935370762650371,
                                           5.841083911459567,
                                           -0.8986440016617555,
                                           -2.9517662582078956,
                                           5.0026753740632515,
                                           -4.905906849591305,
                                           7.847989996961754,
                                           -0.9310582210169116,
                                           2.2781332156064513,
                                           1.979175993616305,
                                           3.2886809041405556,
                                           -0.9606273597635134,
                                           -2.0064036876723,
                                           4.387755267157399,
                                           4.088533270003084,
                                           -8.360162284881437,
                                           0.4618348742121261,
                                           0.016062780287119383,
                                           -3.6988353829878937,
                                           4.739396996105104,
                                           3.8327892474926424,
                                           1.4583759365804085,
                                           3.2554750361193805,
                                           -0.6863185795243316,
                                           -2.0361564497949702,
                                           2.375697644347084,
                                           -1.0226647759803935,
                                           -0.7341151374133368,
                                           -0.525337839865223,
                                           1.829047770157662,
                                           -0.20252436510343086,
                                           5.005504671198587,
                                           -6.254003563847993,
                                           1.119520226839461,
                                           0.4041274926695637,
                                           6.386115029082269,
                                           0.40785148105185254,
                                           -3.6100703211197063,
                                           -1.3411391785820288,
                                           -5.815052117335412,
                                           1.221917945462085,
                                           -8.372200012982317,
                                           -0.8650172306990368,
                                           1.7224633030637484,
                                           -0.6819785587583217,
                                           0.0,
                                           -1.5590584882816314,
                                           0.0,
                                           0.0042756339138692034,
                                           6.307644672640099,
                                           8.773288996733527,
                                           -1.4889535227874742,
                                           3.1394560335760437,
                                           3.9506668467261354,
                                           -2.3411123592187657,
                                           1.69329033505479,
                                           0.0,
                                           4.505463965335339,
                                           -4.793207203672118,
                                           -8.959847454473886,
                                           0.2963783561939113,
                                           2.490154930097335,
                                           -2.9772991980786427,
                                           -0.10258805242017521,
                                           2.515040437144264,
                                           -0.323752470193159,
                                           -1.3871022557850115,
                                           0.09637447265227139,
                                           7.854363178840153,
                                           3.96354237781416,
                                           -0.8107125934130455,
                                           0.7476953283759489,
                                           -0.8673789320239975,
                                           -1.5850387856268093],
                                          [False,
                                           True,
                                           False,
                                           False,
                                           True,
                                           False,
                                           False,
                                           True,
                                           False,
                                           False,
                                           False,
                                           False,
                                           True,
                                           True,
                                           True,
                                           False,
                                           False,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           False,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           True,
                                           False,
                                           True,
                                           False,
                                           False,
                                           True,
                                           False,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           False,
                                           True,
                                           True,
                                           True,
                                           False,
                                           False,
                                           False,
                                           True,
                                           False,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True,
                                           True,
                                           True,
                                           False,
                                           False,
                                           False,
                                           False,
                                           True,
                                           False,
                                           True,
                                           True,
                                           False,
                                           False,
                                           False,
                                           False,
                                           True,
                                           False,
                                           True,
                                           True,
                                           True],
                                          ['JPR',
                                           'EWM',
                                           'WUL',
                                           'VFB',
                                           'EBG',
                                           'PYX',
                                           'SVN',
                                           'ZOH',
                                           'LRA',
                                           'EHI',
                                           'BJO',
                                           'IWL',
                                           'IAB',
                                           'UPM',
                                           'KPH',
                                           'CAV',
                                           'XLJ',
                                           'GVI',
                                           'ZED',
                                           'KMA',
                                           'PMG',
                                           'VQE',
                                           'KZV',
                                           'PDI',
                                           'DKS',
                                           'DVC',
                                           'GZM',
                                           'LXF',
                                           'BTN',
                                           'CXJ',
                                           'KGO',
                                           'MSP',
                                           'QMV',
                                           'CWD',
                                           'LDJ',
                                           'YRD',
                                           'IDF',
                                           'RSC',
                                           'GPB',
                                           'XBD',
                                           'TDA',
                                           'PQM',
                                           'IOY',
                                           'GQT',
                                           'MLN',
                                           'ADN',
                                           'XFY',
                                           'GAE',
                                           'IFY',
                                           'UZJ',
                                           'AHJ',
                                           'NJR',
                                           'LRW',
                                           'GPZ',
                                           'BKJ',
                                           'QZS',
                                           'ISU',
                                           'RXL',
                                           'FUJ',
                                           'PKQ',
                                           'PFI',
                                           'YCR',
                                           'BNA',
                                           'GPU',
                                           'ZJO',
                                           'DUM',
                                           'YES',
                                           'EWI',
                                           'RZO',
                                           'PTV',
                                           'AWF',
                                           'JSF',
                                           'TRV',
                                           'XNA',
                                           'MYX',
                                           'CUQ',
                                           'BCU',
                                           'QNB',
                                           'AYL',
                                           'XBF',
                                           'WPZ',
                                           'WEM',
                                           'XIS',
                                           'PDK',
                                           'WTC',
                                           'ZMC',
                                           'VDL',
                                           'MOC',
                                           'ENL',
                                           'WSE',
                                           'TKZ',
                                           'LGU',
                                           'KVC',
                                           'DEG',
                                           'GSH',
                                           'XBV',
                                           'LMH',
                                           'YFU',
                                           'UEO',
                                           'KNG']],
                                 'schema': {'fields': [{'name': 'numcol_1',
                                                        'type': 'integer'},
                                                       {'name': 'numcol_2',
                                                        'type': 'number'},
                                                       {'name': 'numcol_3',
                                                        'type': 'number'},
                                                       {'name': 'bool_col',
                                                        'type': 'boolean'},
                                                       {'name': 'str_col',
                                                        'type': 'string'}],
                                            'pandas_version': '1.4.0'}}}
```